# Introduction
In this project, I aim to verify the accuracy of point-to-point mileage data provided by airlines for a large financial client. My objective is to calculate true distances between major destination pairs and compare these values to those quoted by carriers to identify potential discrepancies.



## Data Loading
In this section, I will load the dataset and inspect its structure to understand the available columns, data types, and any initial observations regarding data quality.


In [2]:
import pandas as pd

file_path = 'FlightDistanceTest.csv'
flight_data = pd.read_csv(file_path)

flight_data.head()

,Normalised City Pair,CITY1,CITY2,unique departure city,unique arrival city,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon
0,"London, United Kingdom - New York, United Stat...","London, United Kingdom","New York, United States Of America","London, United Kingdom","New York, United States Of America",LHR,JFK,51.5,-0.45,40.64,-73.79
1,"Johannesburg, South Africa - London, United Ki...","Johannesburg, South Africa","London, United Kingdom","Johannesburg, South Africa","London, United Kingdom",JNB,LHR,-26.1,28.23,51.47,-0.45
2,"London, United Kingdom - New York, United Stat...","London, United Kingdom","New York, United States Of America","Bombay, India","Singapore, Singapore",LHR,JFK,51.5,-0.45,40.64,-73.79
3,"Johannesburg, South Africa - London, United Ki...","Johannesburg, South Africa","London, United Kingdom","Dubai, United Arab Emirates","Newark, United States Of America",JNB,LHR,-26.1,28.23,51.47,-0.45
4,"London, United Kingdom - Singapore, Singapore","London, United Kingdom","Singapore, Singapore","Delhi, India","San Francisco, United States Of America",SIN,LHR,1.3,103.98,51.47,-0.45


## Initial Data Exploration
To begin my analysis, I'll perform a preliminary exploration of the dataset. This includes checking for null values, examining unique entries in key columns, and looking for any obvious inconsistencies.


In [3]:
flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Normalised City Pair   3000 non-null   object 
 1   CITY1                  3000 non-null   object 
 2   CITY2                  3000 non-null   object 
 3   unique departure city  281 non-null    object 
 4   unique arrival city    254 non-null    object 
 5   Departure Code         3000 non-null   object 
 6   Arrival Code           3000 non-null   object 
 7   Departure_lat          3000 non-null   float64
 8   Departure_lon          3000 non-null   float64
 9   Arrival_lat            3000 non-null   float64
 10  Arrival_lon            3000 non-null   float64
dtypes: float64(4), object(7)
memory usage: 257.9+ KB


In [4]:
#Data Preparation in Excel
#In the original CSV file, I performed initial data preparation steps in Excel:
#1. Split the `Normalised City Pair` column into separate columns for each city.
#2. Created lists of unique departure and arrival cities.
#3. Conducted basic data exploration to identify potential errors in the dataset.

## Consistency Check for City Coordinates
In this section, I will verify the consistency of latitude and longitude values for each unique city. The objective is to ensure that each city has a single, correct set of coordinates, which will be essential for accurate distance calculations later on.



### Checking Coordinate Consistency for Unique Cities
To ensure that each unique city has a single set of latitude and longitude coordinates, I will group by `Departure Code` and `Arrival Code` and count the number of distinct latitude-longitude pairs. Cities with more than one unique pair may have inconsistencies that could affect our distance calculations.



In [5]:
unique_departure_cities = flight_data['Departure Code'].unique()
unique_arrival_cities = flight_data['Arrival Code'].unique()

In [6]:
# Displaying the unique values in both columns to check for consistency
unique_departure_cities[:10], unique_arrival_cities[:10]

(array(['LHR', 'JNB', 'SIN', 'JFK', 'LCY', 'EWR', 'DXB', 'DEL', 'DME',
        'SFO'], dtype=object),
 array(['JFK', 'LHR', 'LCY', 'BOM', 'SFO', 'HKG', 'LAX', 'CPT', 'GRU',
        'LIN'], dtype=object))

In [7]:
#Group by 'Departure Code' and aggregate unique values for lat/lon coordinates
departure_coord_check = flight_data.groupby('Departure Code')[['Departure_lat', 'Departure_lon']].nunique()

# Group by 'Arrival Code' and aggregate unique values for lat/lon coordinates
arrival_coord_check = flight_data.groupby('Arrival Code')[['Arrival_lat', 'Arrival_lon']].nunique()

In [8]:
# Display rows where the lat/lon counts are more than 1, indicating potential inconsistencies
departure_coord_inconsistencies = departure_coord_check[(departure_coord_check['Departure_lat'] > 1) | (departure_coord_check['Departure_lon'] > 1)]
arrival_coord_inconsistencies = arrival_coord_check[(arrival_coord_check['Arrival_lat'] > 1) | (arrival_coord_check['Arrival_lon'] > 1)]

departure_coord_inconsistencies, arrival_coord_inconsistencies

(                Departure_lat  Departure_lon
 Departure Code                              
 ABQ                         2              2
 ACC                         4              4
 AMS                         3              3
 ARN                         3              3
 ASE                         2              2
 ...                       ...            ...
 VNO                         4              4
 WAW                         2              2
 YYC                         3              3
 YYZ                         3              3
 ZRH                         3              3
 
 [120 rows x 2 columns],
 Empty DataFrame
 Columns: [Arrival_lat, Arrival_lon]
 Index: [])

In [9]:
# Extract inconsistent departure codes
inconsistent_departure_codes = departure_coord_inconsistencies.index.tolist()

In [10]:
# Filter the original data for rows with inconsistent departure codes
inconsistent_departure_data = flight_data[flight_data['Departure Code'].isin(inconsistent_departure_codes)]

# Group the inconsistent data by 'Departure Code' to show all unique lat/lon pairs for each inconsistent code
departure_inconsistent_pairs = inconsistent_departure_data.groupby(['Departure Code', 'Departure_lat', 'Departure_lon']).size().reset_index(name='Count')
print(departure_inconsistent_pairs)


    Departure Code  Departure_lat  Departure_lon  Count
0              ABQ           35.1        -106.63      1
1              ABQ           39.8        -104.86      1
2              ACC           -1.3          36.81      4
3              ACC            5.6          -0.17      5
4              ACC           25.3          55.35      1
..             ...            ...            ...    ...
404            YYZ           43.7         -79.61     26
405            YYZ           50.1           8.56      1
406            ZRH           47.5           8.57     37
407            ZRH           50.1           8.56      4
408            ZRH           51.5          -0.45      2

[409 rows x 4 columns]


In [11]:
import math

def haversine(lat1, lon1, lat2, lon2, radius=6371, unit="km"):
    """ 
    Calculate the great-circle distance between two points on the Earth's surface.

    Parameters:
    ----------
    lat1 : float
        Latitude of the first point in degrees.
    lon1 : float
        Longitude of the first point in degrees.
    lat2 : float
        Latitude of the second point in degrees.
    lon2 : float
        Longitude of the second point in degrees.
    radius : float, optional
        Radius of the Earth. Default is 6371 km.
    unit : str, optional
        Unit of distance. 'km' for kilometers (default) or 'miles'.

    Returns:
    -------
    float
        Distance between the two points in the specified unit.
    
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    #Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    #distance in kms
    distance_km = radius * c
    
    if unit == "miles":
        return distance_km * 0.621371
    
    return distance_km


In [12]:
#testing the function on a sample pair of coordinates
#used site called: https://www.latlong.net/ to veryfi some of the coordinates
#and also: https://www.distance.to/JFK/LHR to double check the calculations and if my function works correctly. 
sample_distance = haversine(51.5, -0.45, 40.64, -73.79) #London to JFK
print(round(sample_distance,2))

jfk_sfo_test = haversine(40.64, -73.79, 37.61, -122.39)
print(round(jfk_sfo_test))

5540.14
4152


### Filtering Most Frequent Coordinates as Reference Values
Assuming the coordinates with the highest occurrence count are correct, I will filter these for each inconsistent departure code.


In [13]:
# Identify the most frequent coordinates for each inconsistent departure code
correct_departure_coords = departure_inconsistent_pairs.loc[
    departure_inconsistent_pairs.groupby('Departure Code')['Count'].idxmax()
]

In [14]:
# Create a dictionary of correct coordinates based on the most frequent values
correct_coords_dict = correct_departure_coords.set_index('Departure Code')[['Departure_lat', 'Departure_lon']].to_dict('index')

In [15]:
# Define function to retrieve correct coordinates from the dictionary
def get_correct_coords(code):
    """Retrieve correct latitude and longitude for a given departure code."""
    return correct_coords_dict.get(code, {'Departure_lat': None, 'Departure_lon': None})

In [16]:
# Apply correct coordinates for inconsistent codes
flight_data['Correct_Departure_lat'] = flight_data['Departure Code'].apply(lambda x: get_correct_coords(x)['Departure_lat'])
flight_data['Correct_Departure_lon'] = flight_data['Departure Code'].apply(lambda x: get_correct_coords(x)['Departure_lon'])

In [17]:
# Identify and fill correct coordinates for consistently used codes
consistent_departure_coords = flight_data.groupby('Departure Code')[['Departure_lat', 'Departure_lon']].nunique()
consistent_departure_codes = consistent_departure_coords[
    (consistent_departure_coords['Departure_lat'] == 1) & (consistent_departure_coords['Departure_lon'] == 1)
].index.tolist()

In [18]:

for code in consistent_departure_codes:
    mask = (flight_data['Departure Code'] == code) & (flight_data['Correct_Departure_lat'].isnull())
    flight_data.loc[mask, 'Correct_Departure_lat'] = flight_data.loc[mask, 'Departure_lat']
    flight_data.loc[mask, 'Correct_Departure_lon'] = flight_data.loc[mask, 'Departure_lon']

In [19]:
# Calculate reference distance using correct coordinates for departure and arrival
flight_data['Reference_Distance'] = flight_data.apply(
    lambda row: haversine(row['Correct_Departure_lat'], row['Correct_Departure_lon'], row['Arrival_lat'], row['Arrival_lon']),
    axis=1
)

## Conclusion
In this analysis, I thoroughly examined the provided dataset to diagnose and address data inconsistencies. I identified errors, specifically in the departure latitude and longitude values for certain airport codes, which displayed multiple coordinate pairs for the same airport. By evaluating the frequency of each coordinate set and cross-referencing with external sources, I determined the most accurate values for each airport. Using these verified coordinates, I calculated precise point-to-point distances between departure and arrival airports. This final dataset provides reliable distances, which can now be compared effectively with carrier-provided values.



In [ ]:
# Selecting relevant columns for export
export_columns = [
    'Departure Code', 'Correct_Departure_lat', 'Correct_Departure_lon',
    'Arrival Code', 'Arrival_lat', 'Arrival_lon', 
    'Reference_Distance',
]

# Export to CSV
flight_data[export_columns].to_csv('FlightDistanceResults.csv', index=False)


In [26]:
import folium
from folium import PolyLine
import branca

# Select a subset of routes (first 100 rows)
subset_flight_data = flight_data.head(100)

# Creating a colormap based on distance
colormap = branca.colormap.LinearColormap(
    colors=['green', 'yellow', 'red'],
    vmin=flight_data['Reference_Distance'].min(),
    vmax=flight_data['Reference_Distance'].max(),
    caption="Route Distance (km)"
)

# Initialize a map
m = folium.Map(location=[flight_data['Departure_lat'].mean(), flight_data['Departure_lon'].mean()], zoom_start=2, titles="Stamen Watercolor")

# Add routes with colors based on distance and interactive tooltips
for _, row in subset_flight_data.iterrows():
    folium.PolyLine(
        [(row['Departure_lat'], row['Departure_lon']), (row['Arrival_lat'], row['Arrival_lon'])],
        color=colormap(row['Reference_Distance']),
        weight=2,
        opacity=0.8,
        tooltip=f"Route: {row['Departure Code']} -> {row['Arrival Code']}, Distance: {row['Reference_Distance']:.2f} km"
    ).add_to(m)

# Add markers for departure and arrival airports
for _, row in subset_flight_data.iterrows():
    # Departure airport marker
    folium.Marker(
        location=[row['Departure_lat'], row['Departure_lon']],
        popup=f"Departure: {row['Departure Code']}",
        icon=folium.Icon(color="blue", icon="plane")
    ).add_to(m)
    
    # Arrival airport marker
    folium.Marker(
        location=[row['Arrival_lat'], row['Arrival_lon']],
        popup=f"Arrival: {row['Arrival Code']}",
        icon=folium.Icon(color="red", icon="plane")
    ).add_to(m)

#Adding colormap legend to the map
colormap.add_to(m)

#Saving the final map
m.save("EnhancedFlightRoutesMap.html")
m



